In [35]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from ctrace.utils import *
from ctrace.dataset import *
from ctrace.constraint import *
from ctrace.restricted import *

In [47]:
G, rev_nodes = load_graph_cville()
SIR = load_sir("a5.json", merge=True)
S, I, R = SIR["S"], SIR["I"], SIR["R"]


ObjectiveTracker = namedtuple("Restricted", ["MinExposedObjective"])
def restricted_trial(budget, policy, **kwargs):
    # Run to_q
    p = 0.06
    _, sol = to_q(G, I, R, budget, "rweighted", p, rev_nodes, policy)
    V1, V2 = find_excluded_contours(G, I, R)
    # Compute MinExposed Objective
    return min_exposed_objective(G, (S,I,R), (V1, V2), p, sol, trials=2)

def regular_trial(budget, **kwargs):
    # Run to_q
    p = 0.06
    _, sol = to_quarantine(G, I, R, budget, p, "weighted")
    V1, V2 = find_excluded_contours(G, I, R)
    # Compute MinExposed Objective
    return min_exposed_objective(G, (S,I,R), (V1, V2), p, sol, trials=2)

In [48]:
from ctrace.runner import *

config = {
    "budget": [i for i in range(500, 2001 + 1, 500)], # The k value
    "policy": ["equal", "old", "adult"],
}

# in_schema and out_schema MUST match the input arguments and namedtuple respectively!
in_schema = list(config.keys())
out_schema = list(ObjectiveTracker._fields)
run = GridExecutorParallel.init_multiple(config, in_schema, out_schema, func=restricted_trial, trials=10)
# Attempt at making schemas extensible - quite hacky right now
# run.track_duration()
run.exec()

Logging Directory Initialized: /home/zach/Documents/prog/research/ContactTracing/output/run_5HC7F


  0%|          | 0/120 [01:09<?, ?it/s]


KeyboardInterrupt: 